In [70]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas
from to_dataframe import make_dataframe, merged_dataframes, list_of_players
from injury_raport_scraper import get_injury_raport

Create necessary dataframes

In [71]:
df1 = make_dataframe('games')
df2 = make_dataframe('games')
df3 = make_dataframe('games')
df4 = make_dataframe('games')
df5 = make_dataframe('games')
df6 = merged_dataframes()
df7 = get_injury_raport()
df8 = make_dataframe('games')
df9 = make_dataframe('games')
df10 = merged_dataframes()
df11 = merged_dataframes()
players_name = list_of_players()

Make them interactive

In [72]:
idf1 = df1.interactive()
idf2 = df2.interactive()
idf3 = df3.interactive()
idf4 = df4.interactive()
idf5 = df5.interactive()
idf6 = df6.interactive()
idf7 = df7.interactive()
idf8 = df8.interactive()
idf9 = df9.interactive()
idf10 = df10.interactive()
idf11 = df11.interactive()

In [73]:
menu_items = ['ATL','BOS','BKN','CHA','CHI','CLE','DET','IND','MIA','MIL','NYK','ORL','PHI','TOR','WAS','DAL','DEN','GSW','HOU','LAC','LAL','MEM','MIN','NOP','OKC','PHX','POR','SAC','SAS','UTA']

Create select buttons

In [74]:
select_button_1 = pn.widgets.Select(name='Team 1', options=menu_items, width=75, margin=50, value='ATL')

select_button_2 = pn.widgets.Select(name='Team 2', options=menu_items, width=75, margin=50, value='BOS')

Create sliders

In [75]:
last_games_slider = pn.widgets.IntSlider(name='Last games slider', start=1, end=10, step=1, value=5, sizing_mode='scale_both', margin=50)

h2h_games_slider = pn.widgets.IntSlider(name='Head to head games slider', start=1, end=10, step=1, value=5, sizing_mode='scale_both', margin=50)

Create checkbox

In [76]:
checkbox_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=[], options=['fgm','fga','fg_pct','3pm','3pa','3p_pct','ftm','fta','ft_pct','oreb','dreb','reb','ast','stl','blk','tos','pf','pts','plus_minus'],
    inline=False)

Create player search autocomplete input

In [ ]:
autocomplete1 = pn.widgets.AutocompleteInput(
    name='Player 1', options=players_name,
    case_sensitive=False, search_strategy='includes',
    placeholder='Player name',
    sizing_mode='scale_both',
    margin=10)

In [ ]:
autocomplete2 = pn.widgets.AutocompleteInput(
    name='Player 2', options=players_name,
    case_sensitive=False, search_strategy='includes',
    placeholder='Player name',
    sizing_mode='scale_both',
    margin=10)

Create datapipelines that changes dataframes

In [77]:
team1_games_home_pipeline = (
    idf1[
        (idf1['home'] == select_button_1)
    ]
    .sort_values(by='game_date', ascending=False)
    .head(last_games_slider)
)

In [78]:
team1_games_away_pipeline = (
    idf2[
        (idf2['away'] == select_button_1)
    ]
    .reset_index()
    .sort_values(by='game_date', ascending=False)
    .reset_index(drop=True)
    .head(last_games_slider)
)

In [79]:
team2_games_home_pipeline = (
    idf3[
        (idf3['home'] == select_button_2)
    ]
    .reset_index()
    .sort_values(by='game_date', ascending=False)
    .reset_index(drop=True)
    .head(last_games_slider)
)

In [80]:
team2_games_away_pipeline = (
    idf4[
        (idf4['away'] == select_button_2)
    ]
    .reset_index()
    .sort_values(by='game_date', ascending=False)
    .reset_index(drop=True)
    .head(last_games_slider)
)

In [81]:
h2h_pipeline = (
    idf5[
        ((idf5['home'] == select_button_1) & (idf5['away'] == select_button_2)) | ((idf5['home'] == select_button_2) & (idf5['away'] == select_button_1))
    ]
    .reset_index()
    .sort_values(by='game_date', ascending=False)
    .reset_index(drop=True)
    .head(h2h_games_slider)
)

In [82]:
team1_game_detail = (
    idf6[
        (idf6['curr_team'] == select_button_1)
    ]
    .groupby('player')[checkbox_group].mean()
    .round(1)
)

In [83]:
team2_game_detail = (
    idf6[
        (idf6['curr_team'] == select_button_2)
    ]
    .groupby('player')[checkbox_group].mean()
    .round(1)
)

In [ ]:
injury_df = (
    idf7[
        ((idf7['team_id'] == select_button_1) | (idf7['team_id'] == select_button_2))
    ]
    .reset_index()
    .reset_index(drop=True)
)

In [ ]:
team1_games = (
    idf8[
        ((idf8['home'] == select_button_1) |  (idf8['away'] == select_button_1))
    ]
    .reset_index()
    .sort_values(by='game_date', ascending=False)
    .reset_index(drop=True)
    .head(last_games_slider)
)

In [ ]:
team2_games = (
    idf9[
        ((idf9['home'] == select_button_2) |  (idf9['away'] == select_button_2))
    ]
    .reset_index()
    .sort_values(by='game_date', ascending=False)
    .reset_index(drop=True)
    .head(last_games_slider)
)

In [ ]:
player1_stats_pipeline = (
    idf10[
        (idf10['player'] == autocomplete1)
    ]
    .reset_index()
    .sort_values(by='game_date', ascending=False)
    .loc[:, checkbox_group]
    .reset_index(drop=True)
    .head(last_games_slider)
)

In [ ]:
player2_stats_pipeline = (
    idf11[
        (idf11['player'] == autocomplete2)
    ]
    .reset_index()
    .sort_values(by='game_date', ascending=False)
    .loc[:, checkbox_group]
    .reset_index(drop=True)
    .head(last_games_slider)
)

Create plots

In [84]:
team1_plot_games_home = team1_games_home_pipeline.hvplot.bar(x = 'game_date', y = ['pts_home', 'pts_away'], title="Chart of team 1's home games", rot=90)

In [85]:
team1_plot_games_away = team1_games_away_pipeline.hvplot.bar(x = 'game_date', y = ['pts_home', 'pts_away'], title="Chart of team 1's away games", rot=90)

In [86]:
team2_plot_games_home = team2_games_home_pipeline.hvplot.bar(x = 'game_date', y = ['pts_home', 'pts_away'], title="Chart of team 2's home games", rot=90)

In [87]:
team2_plot_games_away = team2_games_away_pipeline.hvplot.bar(x = 'game_date', y = ['pts_home', 'pts_away'], title="Chart of team 2's away games", rot=90)

Create tables

In [96]:
h2h = pn.widgets.Tabulator(h2h_pipeline, hidden_columns =['index', 'game_id'])

In [100]:
df_widget = pn.widgets.Tabulator(team1_game_detail)
df_widget2 = pn.widgets.Tabulator(team2_game_detail)
df_widget3 = pn.widgets.Tabulator(injury_df)
df_widget4 = pn.widgets.Tabulator(team1_games, hidden_columns =['index', 'game_id'])
df_widget5 = pn.widgets.Tabulator(team2_games, hidden_columns =['index', 'game_id'])
df_widget6 = pn.widgets.Tabulator(player1_stats_pipeline)
df_widget7 = pn.widgets.Tabulator(player2_stats_pipeline)

Tabulator(value=Empty DataFrame
Columns: [...)

Create layout

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title = 'Moneyball',
    sidebar_width=250,
    sidebar = [
        pn.pane.PNG('logo.png', sizing_mode='scale_both'),
        pn.pane.Markdown('#### This project was created to develop some python, SQL and data visualisation skills. And also wanted to try my luck in sport betting (NBA to be more precise). '),
        select_button_1,
        select_button_2,
        last_games_slider,
        h2h_games_slider,
        autocomplete1,
        autocomplete2,
        checkbox_group
    ],
    main = [pn.Row(
                pn.Column(
                    team1_plot_games_home.panel()),
                pn.Column(            
                    pn.pane.Str('Team 1 last games',styles={'font-size': '12pt'}),
                    pn.panel(df_widget4))
    ),
            pn.Row(
                pn.Column(
                    team1_plot_games_away.panel()),
                pn.Column(            
                    pn.pane.Str('Head to head last games',styles={'font-size': '12pt'}),
                    pn.panel(h2h))
    ),
            pn.Row(
                pn.Column(
                    team2_plot_games_home.panel()),
                pn.Column(            
                    pn.pane.Str('Team 2 last games',styles={'font-size': '12pt'}),
                    pn.panel(df_widget5))
    ),
            pn.Row(
                pn.Column(
                    team2_plot_games_away.panel()),
                pn.Column(            
                    pn.pane.Str('Injury raport',styles={'font-size': '12pt'}),
                    pn.panel(df_widget3))
    ),
            pn.Row(
                pn.Column(
                    pn.pane.Str('Team 1 players stats',styles={'font-size': '12pt'}),
                    pn.panel(df_widget)),
                pn.Column(
                    pn.pane.Str('Team 2 players stats',styles={'font-size': '12pt'}),
                    pn.panel(df_widget2))
    ),
            pn.Row(
                pn.Column(
                    pn.pane.Str('Players 1 stats last games',styles={'font-size': '12pt'}),
                    pn.panel(df_widget6)),
                pn.Column(
                    pn.pane.Str('Players 2 stats last games',styles={'font-size': '12pt'}),
                    pn.panel(df_widget7))
    )
    ]
)

#show template
template.servable()